In [1]:
import torch
import nltk
from nltk.stem.porter import PorterStemmer
import spacy
import torchtext
from torchtext.data import Field
import numpy as np
import json
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [2]:
#Loading spacy for tokenizing and stemme for stemming of the data
spacy_eng = spacy.load("en_core_web_sm")
stemmer = PorterStemmer()

In [3]:
remove_punc =['?',',','.','_','!']


In [4]:
#Preparing the tokenizer and the stemmer function
def tok_eng(text):
    return [(tok.text) for tok in spacy_eng.tokenizer(text)]
def stemer(text):
    return [stemmer.stem(word) for word in text if word  not in remove_punc]


In [5]:
#Testing the working of the stemer
text = "Organizing the organ in the Human body, and ? "
tex= tok_eng(text)

stemer(tex)


['organ', 'the', 'organ', 'in', 'the', 'human', 'bodi', 'and']

In [6]:
#creating bag of words for model training
 
def bag_of_words(input_words, all_words):
    tokenize = tok_eng(input_words)
    stemmed_words = stemer(tokenize)
    input_array = np.zeros(len(all_words), dtype=np.float32)
    for i, word in enumerate(all_words):
        if word in stemmed_words:
            input_array[i]=1
    return input_array    

In [7]:
#checking out bagging function as it will be the input to the nn
inp = "hi"
out = ['do', 'doe', 'funni', 'get', 'good', 'goodby', 'have', 'hello', 'help', 'hey', 'hi', 'how', 'i', 'is', 'item', 'joke', 'kind', 'know', 'later', 'long', 'lot', 'mastercard', 'me', 'my', 'of', 'onli', 'pay', 'paypal', 'see', 'sell', 'ship', 'someth', 'take', 'tell', 'thank', 'that', 'there', 'what', 'when', 'which', 'with', 'you']

inp = tok_eng(inp)
inp = ' '.join(inp)
print(out)
bag_of_words(inp,out)

['do', 'doe', 'funni', 'get', 'good', 'goodby', 'have', 'hello', 'help', 'hey', 'hi', 'how', 'i', 'is', 'item', 'joke', 'kind', 'know', 'later', 'long', 'lot', 'mastercard', 'me', 'my', 'of', 'onli', 'pay', 'paypal', 'see', 'sell', 'ship', 'someth', 'take', 'tell', 'thank', 'that', 'there', 'what', 'when', 'which', 'with', 'you']


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [8]:
#Loading our json file 
with open('data.json','r') as f:
    data = json.load(f)

In [9]:
#Mapping our datas in in the json file to make the ultimate maapping of tags and it's response for our all words

all_words = []
tags = []
word_tag_map =[]

for datas in data['intents']:
    tag = datas['tag']  
    tags.append(tag)

    for pattern in datas['patterns']:
        stemming = stemer(tok_eng(pattern))
        all_words.extend(stemming)      
        word_tag_map.append((stemming, tag))
        
all_words = sorted(set(all_words))
tags = sorted(set(tags))
tags

['availability',
 'capabilities',
 'delivery',
 'funny',
 'goodbye',
 'greeting',
 'items',
 'payments',
 'personal_info',
 'thanks']

In [10]:
my_list = ['Hello', 'World', '!']

# Convert the list to a string with a space (' ') separator
my_string = ' '.join(my_list)

print(my_string)

Hello World !


In [11]:
# create training data
X_train = []
y_train = []
for (pattern_sentence, tag) in word_tag_map:
    # X: bag of words for each pattern_sentence
    pattern_sentence = ' '.join(pattern_sentence)
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag)
    y_train.append(label)
print(y_train[1] ) 
(X_train[1])


5


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [12]:
X_train = np.array(X_train)
y_train = np.array(y_train)
print(len(X_train),len(y_train))

35 35


In [13]:
#creating our dataset

class Bot(Dataset):
    def __init__(self):
        self.samples = len(X_train)
        self.inp = X_train
        self.output = y_train

    def __getitem__(self, index):
        return self.inp[index], self.output[index]
    
    def __len__(self):
        return self.samples
dataset = Bot()

In [14]:
#creating a neural network for training of only 3 layers

class ChatBot(nn.Module):
    def __init__(self, inp_size, hidden_size, output_size) :
        super(ChatBot,self).__init__()
        self.input_size = nn.Linear(inp_size, hidden_size)
        self.hidden_size = nn.Linear(hidden_size, hidden_size)
        self.output_size=nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

    def forward(self,x):
        x1= self.input_size(x)
        x1=self.relu(x1)
        x1= self.hidden_size(x1)
        x1=self.relu(x1)
        x1= self.output_size(x1)
        return x1



In [15]:
# Hyper-parameters 
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 10
output_size = len(tags)
print(input_size, output_size)


66 10


In [16]:
#Using dataloader for subjecting into training

train_loader = DataLoader(dataset=dataset,
                          batch_size=8,
                          shuffle=True)
test = next(iter(train_loader))
test


[tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
          1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
          0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   

In [17]:
from tqdm import tqdm
device = 'cuda' if torch.cuda.is_available else 'cpu'

In [18]:
model = ChatBot(input_size, hidden_size, output_size ).to(device)
model

ChatBot(
  (input_size): Linear(in_features=66, out_features=10, bias=True)
  (hidden_size): Linear(in_features=10, out_features=10, bias=True)
  (output_size): Linear(in_features=10, out_features=10, bias=True)
  (relu): ReLU()
)

In [19]:
#setting the optim and lossfun
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [24]:
#Now training our model
for epoch in tqdm(range(num_epochs)):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
#         # Forward pass
        outputs = model(words)    
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if epoch%100 ==0:
        print(f"[Epoch {epoch} / {num_epochs}], Loss: {loss.item():.4f}")
        checkpoint = {"state_dict": model.state_dict(), "optimizer": optimizer.state_dict(), "epoch": epoch}
        print("===> saving")
        torch.save(checkpoint, "model.pth")


print(f'final loss: {loss.item():.4f}')




  1%|          | 12/1000 [00:00<00:17, 58.02it/s]

[Epoch 0 / 1000], Loss: 0.0003
===> saving


 11%|█         | 106/1000 [00:01<00:13, 67.38it/s]

[Epoch 100 / 1000], Loss: 0.0006
===> saving


 21%|██▏       | 214/1000 [00:03<00:11, 70.97it/s]

...

[Epoch 800 / 1000], Loss: 0.0001
===> saving


 91%|█████████ | 910/1000 [00:13<00:01, 69.59it/s]

[Epoch 900 / 1000], Loss: 0.0000
===> saving


100%|██████████| 1000/1000 [00:15<00:00, 66.14it/s]

final loss: 0.0001


In [25]:
import random

In [26]:
#Testing our model

torch.load('model.pth')
model.eval()

bot_name = "Jarvis"
print("Let's chat! (type 'quit' to exit)")
print("Hi I'm jarvis, Ask me anything?")
while True:
    # sentence = "do you use credit cards?"
    sentence = input("You: ")
    print(f"Me: {sentence}")
    if sentence == "quit":
        break

    # sentence = tok_eng(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]


    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.75:
        for intent in data['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: I do not understand...")


Let's chat! (type 'quit' to exit)
Hi I'm jarvis, Ask me anything?
Me: hello
Jarvis: Hello, thanks for visiting
Me: hi
Jarvis: Hi there, how can I help?
Me: what is your name
Jarvis: See you later, thanks for visiting
Me: what is your name
Jarvis: See you later, thanks for visiting
Me: name
Jarvis: See you later, thanks for visiting
Me: when are you available
Jarvis: I am available 24/7 to assist you.
Me: ??
Jarvis: Bye! Come back again soon.
Me: slow
Jarvis: Have a nice day
Me: hello
Jarvis: Hey :-)
Me: what are your service
Jarvis: I do not understand...
Me: what service can you give me
Jarvis: My capabilities include answering queries, providing information, and offering assistance on various topics.
Me: avaibility
Jarvis: Have a nice day
Me: s
Jarvis: See you later, thanks for visiting
Me: quit
